---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [3]:
import pandas as pd
import numpy as np
import re
def getPowerTechnology():
    ScimEn = pd.read_excel("scimagojr-3.xlsx")
    ScimEn = ScimEn.set_index('Country')
    return ScimEn
def replaceWord(row):
    val = row['Country']
    #print("Val: {}".format(val))
    if pd.isnull(val)==False:
        new_text = re.sub("\(\s*[\w*\s*]*\s*\)","",row['Country'])
        new_text = re.sub("[\.]+","",new_text.strip())
        new_text = re.sub("[\d]+","",new_text.strip())
        #print(new_text)
        row['Country'] = new_text
    return row
def putBackNaN(row):
    filtered_row = row[row['Country']==""]
    filtered_row = np.NaN
    return row
def getEnergyDataFrame():
    df = pd.read_excel("Energy Indicators.xls",skiprows = [i for i in range(17)],skip_footer=1)
    #['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    df = df.drop([df.columns[0],df.columns[1]],axis=1)
    df = df.rename(columns={df.columns[0]:'Country',df.columns[1]:'Energy Supply',
                            df.columns[2]:'Energy Supply per Capita',df.columns[3]:'% Renewable'})
    df.fillna(value=np.NaN)
    df['Country'] = df['Country'].fillna(value="")
    #not_null  = df["Energy Supply"].notnull()
    df.ix[(df["Energy Supply"].notnull) & (df["Energy Supply"].str.contains("\.+")),"Energy Supply"] = np.NaN
    df.ix[(df["Energy Supply per Capita"].notnull) & (df["Energy Supply per Capita"].str.contains("\.+")),"Energy Supply per Capita"] = np.NaN
    #df.ix[(df["% Renewable"].notnull) & (df["% Renewable"].str.contains("\.+")),"% Renewable"] = np.NaN
    #df[df["Energy Supply per Capita"].str.contains("\.+")] = np.NaN
    #df[df["% Renewable"].str.contains("\.+")] = np.NaN
    df = df.apply(replaceWord,axis=1)
    #df.apply(putBackNaN,axis=1)
    df = df.set_index('Country')
    df = df.loc["Afghanistan":"Zimbabwe":]
    df = df.rename(index = {"Republic of Korea": "South Korea","United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"})
    df["Energy Supply"] = df["Energy Supply"]*1000000
    
    #df = df.ix["India":"Ireland",:]
    #df = df.dropna()
    #print(df.shape)
    return df
    #return energy.tail(50)
    
def dealwithgdp():
    GDP = pd.read_csv("world_bank.csv",skiprows=[0,1,2,3])
    GDP = GDP.set_index("Country Name")
    GDP = GDP.rename(index={"Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"})
    GDP = GDP.ix[:,"2006":]
    #GDP = pd.concat([GDP.ix[:,"2006":],GDP.ix[:,'Country Name']],axis=1)
    #print(GDP.shape)
    return GDP
def mergeDataFrames():
    tot_rows = 0
    GDP = dealwithgdp()
    r,c = GDP.shape
    tot_rows += r
    Energy = getEnergyDataFrame()
    r,c = Energy.shape
    tot_rows += r
    ScimEn = getPowerTechnology()
    r,c = ScimEn.shape
    tot_rows += r
    df = pd.merge(ScimEn,Energy,how="inner",left_index=True,right_index=True)
    r,c = df.shape
    tot_rows -= r
    df1 = pd.merge(ScimEn,GDP,how="inner",left_index=True,right_index=True)
    r,c = df1.shape
    tot_rows -= r
    df2 = pd.merge(GDP,Energy,how="inner",left_index=True,right_index=True)
    r,c = df2.shape
    tot_rows -= r
    df = pd.merge(df,GDP,how="inner",left_index=True,right_index=True)
    #r,c = df.shape
    #tot_rows -= r
    
    return (df,tot_rows)
def answer_one():
    df,tot_rows = mergeDataFrames()
    #print(df.shape)
    df = df.sort_values('Rank',ascending=True)
    return df.head(15)
    #return ScimEn
answer_one()
#getEnergyDataFrame()
#dealwithgdp()
#getPowerTechnology()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [4]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [5]:
def answer_two():
    df,tot_rows = mergeDataFrames()
    return tot_rows
answer_two()

156

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [6]:
def averageGDP(row):
    row['avgGDP'] = np.nanmean(row)
    #print("Row: {}\n".format(row))
    return row
    
                      
def answer_three():
    Top15 = answer_one()
    Top15 = Top15.ix[:,"2006":"2015"]
    #Top15['avgGDP'] = np.NaN
    Top15 = Top15.apply(averageGDP,axis=1)
    s = pd.Series(Top15['avgGDP'],index = Top15.index)
    #print(s)
    #s = s.sort_values(axis=1,ascending = False)
    s = s.sort_values(ascending = False)
    '''
    Top15 = Top15.ix[:,"2006":"2015"]
    Top15['avgGDP'] = np.average(Top15.ix[:,"2006":"2015"])
    Top15 = Top15.groupby(level=0)["2006","2007"].agg({'meanGdp':np.average})
    '''
    #return Top15.head()
    return s
answer_three()

United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [7]:
def answer_four():
    Top15 = answer_one()
    sixthLargest = answer_three().index[5]
    #print(sixthLargest)
    initial = Top15.loc[sixthLargest]["2006"]
    final = Top15.loc[sixthLargest]["2015"]
    change = ((final-initial)/initial)
    #print("Initial: {} Final {} Change {}".format(initial,final,change))
    return change
answer_four()

0.10195882207745173

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [8]:
def answer_five():
    Top15 = answer_one()
    meanEnrg = np.mean(Top15.ix[:,"Energy Supply per Capita"])
    return meanEnrg
answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [9]:
def answer_six():
    Top15 = answer_one()
    maxRenewable = np.max(Top15.ix[:,"% Renewable"])
    maxRow = Top15[Top15.ix[:,"% Renewable"]==maxRenewable]
    return maxRow.index[0],maxRenewable
answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [10]:
def answer_seven():
    Top15 = answer_one()
    Top15['Citation Ratio'] = Top15.ix[:,"Self-citations"] / Top15.ix[:,"Citations"]
    maxRatio = np.max(Top15['Citation Ratio'])
    highestCountry = Top15[Top15['Citation Ratio']==maxRatio].index[0]
    return highestCountry,maxRatio
answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [11]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15 = Top15.sort_values(['Population'],axis=0,ascending = False)
    return Top15.index[2]
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [12]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    df = pd.concat([Top15['Energy Supply per Capita'],Top15['Citable docs per Capita']],axis=1)
    
    return df.corr(method='pearson').ix[0:1,"Citable docs per Capita"][0]
answer_nine()

0.79400104354429435

In [13]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [14]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [15]:
def answer_ten():
    Top15 = answer_one()
    median = np.median(Top15['% Renewable'])
    #print(median)
    Top15['HighRenew'] = -1
    #print("Median: {} ".format(Top15['% Renewable']>=median))
    Top15['HighRenew'][Top15['% Renewable']>=median] = 1
    Top15['HighRenew'][Top15['% Renewable']<median] = 0
    s = pd.Series(Top15['HighRenew'],index = Top15.index)
    return s
answer_ten()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [16]:
def assignContinent(row):
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    country = row["Country"]
    continent = ContinentDict[country]
    row["Continent"] = continent
    #print(country,continent)
    return row
    
def answer_eleven():
    Top15 = answer_one()
    Top15["Country"] = Top15.index
    Top15 = Top15.apply(assignContinent,axis=1)
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = pd.concat([Top15['PopEst'],Top15["Continent"]],axis=1)
    #'size':'count','sum':np.sum,'mean':np.mean,
    groupped = Top15.groupby('Continent')
    return groupped['PopEst'].agg({'size':'count','sum':np.sum,'mean':np.mean,'std':np.std})
answer_eleven()

,sum,std,size,mean
Continent,,,,
Asia,2.898666e+09,6.790979e+08,5,5.797333e+08
Australia,2.331602e+07,NaN,1,2.331602e+07
Europe,4.579297e+08,3.464767e+07,6,7.632161e+07
North America,3.528552e+08,1.996696e+08,2,1.764276e+08
South America,2.059153e+08,NaN,1,2.059153e+08


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [17]:
def answer_twelve():
    Top15 = answer_one()
    Top15["Country"] = Top15.index
    Top15 = Top15.apply(assignContinent,axis=1)
    #Top15 = pd.concat([pd.cut(Top15['% Renewable'],5),Top15],axis=1)
    #Top15 = Top15.drop(['Country'],axis=1)
    #Top15 = Top15.set_index(['Continent',pd.cut(Top15['% Renewable'],5)])
    return Top15.groupby(['Continent',pd.cut(Top15['% Renewable'],5)]).agg(sum).dropna()
    #return Top15
answer_twelve()

Rank  Documents  Citable documents  Citations  \
Continent     % Renewable                                                       
Asia          (2.212, 15.753]   34.0    66388.0            65870.0   523932.0   
              (15.753, 29.227]   1.0   127050.0           126767.0   597237.0   
Australia     (2.212, 15.753]   14.0     8831.0             8725.0    90765.0   
Europe        (2.212, 15.753]    4.0    20944.0            20357.0   206091.0   
              (15.753, 29.227]  21.0    48714.0            48105.0   305464.0   
              (29.227, 42.701]  23.0    20392.0            20124.0   235186.0   
North America (2.212, 15.753]    2.0    96661.0            94747.0   792274.0   
              (56.174, 69.648]   6.0    17899.0            17620.0   215003.0   
South America (56.174, 69.648]  15.0     8668.0             8596.0    60702.0   

                                Self-citations  Citations per document  \
Continent     % Renewable                                                
Asia          (2.212, 15.753]         140483.0                   31.92   
              (15.753, 29.227]        411683.0                    4.70   
Australia     (2.212, 15.753]          15606.0                   10.28   
Europe        (2.212, 15.753]          37874.0                    9.84   
              (15.753, 29.227]         68449.0                   20.04   
              (29.227, 42.701]         50625.0                   23.28   
North America (2.212, 15.753]         265436.0                    8.20   
              (56.174, 69.648]         40930.0                   12.01   
South America (56.174, 69.648]         14396.0                    7.00   

                                H index  Energy Supply  \
Continent     % Renewable                                
Asia          (2.212, 15.753]     425.0   7.235800e+10   
              (15.753, 29.227]    138.0   1.271910e+11   
Australia     (2.212, 15.753]     107.0   5.386000e+09   
Europe        (2.212, 15.753]     139.0   7.920000e+09   
              (15.753, 29.227]    297.0   5.456700e+10   
              (29.227, 42.701]    221.0   1.145300e+10   
North America (2.212, 15.753]     230.0   9.083800e+10   
              (56.174, 69.648]    149.0   1.043100e+10   
South America (56.174, 69.648]     86.0   1.214900e+10   

                                Energy Supply per Capita  % Renewable  \
Continent     % Renewable                                               
Asia          (2.212, 15.753]                      515.0    33.188974   
              (15.753, 29.227]                      93.0    19.754910   
Australia     (2.212, 15.753]                      231.0    11.810810   
Europe        (2.212, 15.753]                      124.0    10.600470   
              (15.753, 29.227]                     545.0    52.210490   
              (29.227, 42.701]                     215.0    71.635820   
North America (2.212, 15.753]                      286.0    11.570980   
              (56.174, 69.648]                     296.0    61.945430   
South America (56.174, 69.648]                      59.0    69.648030   

                                        2006          2007          2008  \
Continent     % Renewable                                                  
Asia          (2.212, 15.753]   8.093008e+12  8.409397e+12  8.436388e+12   
              (15.753, 29.227]  3.992331e+12  4.559041e+12  4.997775e+12   
Australia     (2.212, 15.753]   1.021939e+12  1.060340e+12  1.099644e+12   
Europe        (2.212, 15.753]   2.419631e+12  2.482203e+12  2.470614e+12   
              (15.753, 29.227]  7.326523e+12  7.615056e+12  7.736451e+12   
              (29.227, 42.701]  3.616992e+12  3.702773e+12  3.695684e+12   
North America (2.212, 15.753]   1.479230e+13  1.505540e+13  1.501149e+13   
              (56.174, 69.648]  1.564469e+12  1.596740e+12  1.612713e+12   
South America (56.174, 69.648]  1.845080e+12  1.957118e+12  2.056809e+12   

                                        2009 

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [18]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['PopEst'] = Top15['PopEst'].apply(lambda x: '{:,}'.format(x))
    return Top15['PopEst']
answer_thirteen()

China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [19]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!